In [ ]:
import sys
sys.path.append('/juno/work/isabl/home/zhouy1/mergeSVvcf')
import mergesvvcf
import gzip
import pysam
import pandas as pd
import subprocess

In [ ]:
with open("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/samples.tsv",'r') as f:
    samples = f.readlines()
    samples = [i.strip('\n') for i in samples]

    
for sample in samples:
    print(sample)
    brass = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/{sample}_vs_I-H-108298-N1-1-D1-1.annotated.flagged.output.tsv"
    brass_vcf = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/merged_svs.all.vcf.gz"
    svaba = f"/work/isabl/home/gutierj2/uk_all/svABA_unmatch/no_germline/{sample}.svaba.sv.vcf.gz"

    # filter svaba calls by # supporting reads, mapping quality
    svaba = pysam.VariantFile(svaba, 'r')
    header = svaba.header.__str__()

    variants_to_keep = []
    for v in svaba:
        if (v.info['MAPQ']>30) & (v.samples[0]['AD']>10):
            variants_to_keep += [v]
    svaba_filtered = f'/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/{sample}.svaba.filtered.vcf'
    with open(svaba_filtered, 'a') as f:
        f.write(header)
        for i in variants_to_keep:
            f.write(i.__str__())
            
    # filter brass calls by cat1, convert tsv to vcf
    brass = pd.read_csv(brass, sep='\t')
    brass = brass[brass['AnnotSV type'] == 'full']
    brass = brass[brass['cat1_filters'] == 0]
    brass = brass[brass['brass_score'] > 10]
    
    
    with gzip.open(brass_vcf,'rb') as f:
        content = f.readlines()
    content = [i.decode('ascii') for i in content]
    variants = [i for i in content if not i.startswith('#')]
    header = [i for i in content if i.startswith('#')]
    
    filtered_brass = f'/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/{sample}.brass.filtered.vcf'
    with open(filtered_brass,'w') as out:
        out.write(''.join(header))

        counter = 0
        for index, row in brass.iterrows():
            start = row['end1']
            end = row['end2']
            for v in variants:
                v_end = v.split('\t')[4].split(':')[1].strip('N').strip('[').strip(']')
                v_start = v.split('\t')[1]
                if (int(v_end) == int(end)) & (int(start) == int(v_start)):
                    out.write(v)
                if (int(v_end) == int(start)) & (int(end) == int(v_start)):
                    out.write(v)
    
    sorted_brass = f'/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/{sample}.brass.sorted.vcf'
    cmd = ['vcf-sort', filtered_brass]
    f = open(sorted_brass, "w")
    subprocess.check_call(cmd, stdout=f)

    # intersection between two vcfs
    out = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/merged/{sample}.merged.vcf"
    mergesvvcf.mergedfile.merge(
        filenames=[sorted_brass, svaba_filtered],
        programs=['brass','svaba'],
        forceSV=False,
        outfile=out,
        slop=100,
        min_num_callers=2,
        verbose=False
    )

In [ ]:
for sample in samples:
    with open(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/merged/{sample}.merged.vcf", 'r') as fin:
        with open(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/merged/two_callers/{sample}.consensus.vcf",'a') as fout:
            
            content = fin.readlines()
            variants = [i for i in content if not i.startswith('#')]
            header = [i for i in content if i.startswith('#')]
            
            fout.write(''.join(header))

            for v in variants:
                if ('Callers=svaba,brass' in v) & ('PASS' in v):
                    fout.write(v)


In [ ]:
    sample = "IID_H158671_T01_01_WG01"
    brass = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/{sample}_vs_I-H-108298-N1-1-D1-1.annotated.flagged.output.tsv"
    brass_vcf = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/merged_svs.all.vcf.gz"
    svaba = f"/work/isabl/home/gutierj2/uk_all/svABA_unmatch/no_germline/{sample}.svaba.sv.vcf.gz"

    # filter svaba calls by # supporting reads, mapping quality
    svaba = pysam.VariantFile(svaba, 'r')
    header = svaba.header.__str__()

    variants_to_keep = []
    for v in svaba:
        if (v.info['MAPQ']>30) & (v.samples[0]['AD']>10):
            variants_to_keep += [v]
    svaba_filtered = f'/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/{sample}.svaba.filtered.vcf'
    with open(svaba_filtered, 'a') as f:
        f.write(header)
        for i in variants_to_keep:
            f.write(i.__str__())
            
    # filter brass calls by cat1, convert tsv to vcf
    brass = pd.read_csv(brass, sep='\t')
    brass = brass[brass['AnnotSV type'] == 'full']
    brass = brass[brass['cat1_filters'] == 0]
    
    brass

In [ ]:
pd.options.display.max_columns=None
filtered_brass

In [ ]:
# intersection between two vcfs
for sample in samples:
    
    matched_brass = "/work/isabl/data/analyses/21/37/262137/IID_H158671_T01_01_WG01_vs_IID_H158671_N01_01_WG01.annot.vcf.gz"
    unmatched_consensus = "/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/merged/two_callers/IID_H158671_T01_01_WG01.consensus.vcf"
    out = "/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/test.vcf"
    mergesvvcf.mergedfile.merge(
        filenames=[matched_brass, unmatched_consensus],
        programs=['matched_brass','unmatched_consensus'],
        forceSV=False,
        outfile=out,
        slop=100,
        min_num_callers=2, 
        verbose=False
    )
    


In [ ]:
import isabl_cli as ii

In [ ]:
ii.get_analyses(targets_system__id='IID_H158671_T01_01_WG01', application__name='BRASS' )

In [ ]:
with open("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/samples.tsv",'r') as f:
    samples = f.readlines()
    samples = [i.strip('\n') for i in samples]

for sample in samples:
    print(sample)
    gridss = f"/work/isabl/home/zhouy1/gridss/{sample}_unmatched.sv.vcf"
    brass = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/merged_svs.all.vcf.gz"
    svaba = f"/work/isabl/home/gutierj2/uk_all/svABA_unmatch/no_germline/{sample}.svaba.sv.vcf.gz"

    out = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/{sample}.unmatched.merged.vcf"
    mergesvvcf.mergedfile.merge(
        filenames=[gridss, brass, svaba],
        programs=['gridss', 'brass', 'svaba'],
        forceSV=False,
        outfile=out,
        slop=100,
        min_num_callers=2, 
        verbose=False
    )


In [8]:

import tempfile
import os
import glob
import subprocess
import shutil
import gzip

tmps = {
    "IID_H158636_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmp6zlaq2/",
    "IID_H158609_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmpmxZiK9/",
    "IID_H158646_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmpj5lcMC/",
    "IID_H158671_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmprgxw9P/",
    "IID_H158640_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmpU5uN8W/",
    "IID_H158628_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmpfDbKJX/",
    "IID_H158625_T01_01_WG01": "/work/isabl/tmp/gutierj2/2020/02/10/tmpEBt4Go/"
}

root_working_dir = "/work/isabl/home/zhouy1/tmp/tmp_flag"

for sample, tmp in tmps.items():
    working_dir = os.path.join(root_working_dir, sample)

    unsorted_merged_vcf = tempfile.NamedTemporaryFile(
        prefix="unsorted", suffix=".vcf", dir=working_dir, delete=True
    )

    result = []

    for vcf in glob.glob(os.path.join(working_dir, "*.gz")):
        print(vcf)
        with gzip.open(vcf, "r") as f:
            lines = f.readlines()
            headers = [l.decode() for l in lines if l.startswith(b"#")]
            variants = [l.decode() for l in lines if not l.startswith(b"#")]
        result.extend(variants)

    with open(unsorted_merged_vcf.name, "w") as f:
        f.write("".join(headers))
        f.write("".join(result))


    out_vcf = os.path.join(working_dir, "merged.vcf")
    cmd = ["bcftools", 'sort', '-o', out_vcf, unsorted_merged_vcf.name]
    subprocess.check_call(cmd)
    subprocess.check_call(["bgzip", "-f", out_vcf])
    subprocess.check_call(["tabix", "-f", "-p", "vcf", out_vcf + ".gz"])
    unsorted_merged_vcf.close()
    # for vcf in os.listdir(working_dir):
    #     os.remove(vcf)

Exception ignored in: <bound method _TemporaryFileCloser.__del__ of <tempfile._TemporaryFileCloser object at 0x7f3c95815c18>>
Traceback (most recent call last):
  File "/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/tempfile.py", line 450, in __del__
    self.close()
  File "/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/tempfile.py", line 446, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/unsortedvyy6pz45.vcf'


/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_UDIRc4.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_sAWukP.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_r6DBGo.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_fNrHFH.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_FYad1X.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_TLvvcD.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_yZiFpY.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_C6sclc.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_dwxyLs.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_47az1_.flagged.vcf.gz
/work/isabl/home/zhouy1/tmp/tmp_flag/IID_H158640_T01_01_WG01/split_Wh4PVr.flagged.vcf.gz
/work/isabl/home/zhou

In [ ]:
print(1)

In [ ]:
from pysam import VariantFile

vcf = pysam.VariantFile("/work/isabl/home/zhouy1/dbsnp_138.b37_common.vcf.gz")

v = vcf.fetch('1', 10582, 10584)
for i in v:
    print(i)
    

In [ ]:
from pysam import VariantFile


dbvcf_in=VariantFile("/ifs/work/leukgen/ref/homo_sapiens/GRCh37d5/dbsnp/138/dbsnp_138.b37.vcf.gz", 'rb')
dbvcf_out=VariantFile('/work/isabl/home/zhouy1/dbsnp_138.b37_common.vcf', 'w', header=dbvcf_in.header)
for row in dbvcf_in:
    split = row.__str__().split('\t')
    if('COMMON=1' in split[7]):
        if('<' not in split[4]):
            #print(row)
            dbvcf_out.write(row)

In [ ]:
import pandas as pd
with open("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/samples.tsv",'r') as f:
    samples = f.readlines()
    samples = [i.strip('\n') for i in samples]
    
for sample in samples:
    print(sample)
    df = pd.read_csv(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/{sample}_vs_I-H-108298-N1-1-D1-1.annotated.flagged.output.tsv", sep='\t')
    df[df['cat1_filters']!=0].to_csv(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/matthieu_eliminated/{sample}.matthieu.filtered.tsv", sep='\t')

In [ ]:
x-1

In [122]:

with open("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/samples.tsv",'r') as f:
    samples = f.readlines()
    samples = [i.strip('\n') for i in samples]
    
outdir = "/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge"
for sample in samples:
    cmd = [
        'bsub',
        '-e', os.path.join(outdir, f"{sample}_err"),
        '-o', os.path.join(outdir, f"{sample}_out"),
        "python", f"/work/isabl/home/zhouy1/notebook/Unmatched_genome/merge_triple_callers_and_annotate.py", sample
    ]
    subprocess.check_output(cmd)

In [3]:
from bsub import bsub

In [14]:
bsub

bsub.bsub.bsub

In [1]:
import pandas as pd

sample = "IID_H158671_T01_01_WG01"
tsv = f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/{sample}/{sample}_vs_I-H-108298-N1-1-D1-1.annotated.flagged.output.tsv"
df = pd.read_csv(tsv, sep='\t')
filtered = df[df['cat1_filters']!=0]
filtered

,chr1,start1,end1,chr2,start2,end2,SV length,SV type,ID,REF,...,delly_hgsvc_hg38_filtered,DGV_filtered,not_MAPQ1>6,not_MAPQ2>6,not_MATCH1>50,not_MATCH2>50,not_SUPP1<1.75*depth,not_SUPP2<1.75*depth,cat2_filters,cat1_filters
0,1,4480433,4480434,10,7011114,7011115,NaN,TRA,1_4480434_10_7011115_TRA_++,N,...,no,no,no,no,no,no,no,no,1.0,1.0
1,1,4480433,4480434,10,7011114,7011115,NaN,TRA,1_4480434_10_7011115_TRA_++,N,...,no,no,no,no,no,no,no,no,1.0,1.0
4,1,10877921,10877922,6,134717443,134717444,NaN,TRA,1_10877922_6_134717444_TRA_+-,N,...,no,no,yes,no,yes,no,no,no,2.0,2.0
5,1,11773065,11773066,1,27357132,27357133,15584067.0,INV,1_11773066_1_27357133_INV_--,N,...,no,no,no,no,no,no,no,no,0.0,1.0
6,1,11773065,11773066,1,27357132,27357133,15584067.0,INV,1_11773066_1_27357133_INV_--,N,...,no,no,no,no,no,no,no,no,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,X,134030783,134030784,2,34643858,34643859,NaN,TRA,2_34643859_X_134030784_TRA_+-,N,...,no,no,no,yes,no,no,no,no,1.0,1.0
6625,X,155090038,155090039,6,20168886,20168887,NaN,TRA,6_20168887_X_155090039_TRA_+-,N,...,no,no,yes,no,no,no,no,no,2.0,3.0
6626,Y,21204135,21204136,11,65523675,65523676,NaN,TRA,11_65523676_Y_21204136_TRA_--,N,...,no,no,no,yes,no,no,no,no,1.0,1.0
6627,Y,21204135,21204136,11,65523675,65523676,NaN,TRA,11_65523676_Y_21204136_TRA_--,N,...,no,no,no,yes,no,no,no,no,1.0,1.0


In [2]:
with open("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/merged/IID_H158671_T01_01_WG01.merged.vcf", 'r') as f:
    content = f.readlines()
    sg = [i for i in content if "Callers=svaba,gridss" in i]
    sg

In [91]:
# Integrate CNV into SV
sample = "IID_H158606_T01_01_WG01"

with open("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/svaba/samples.tsv",'r') as f:
    samples = f.readlines()
    samples = [i.strip('\n') for i in samples]

for sample in samples:
    print(sample)
    # sv = pd.read_csv("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_sv_1213/IID_H158604_T01_01_WG01/IID_H158604_T01_01_WG01_vs_I-H-108298-N1-1-D1-1.annotated.flagged.output.tsv", sep='\t')
    sv = pd.read_csv(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/annotated/{sample}/{sample}.merged.annotated.tsv", sep='\t')
    sv = sv[sv['AnnotSV type']=='full']
    sv['SV chrom'] = sv['SV chrom'].astype(str)
    sv = sv[(sv['SV type']=='DEL') | (sv['SV type']=='DUP')]
    for index, row in sv.iterrows():
        INFO = row['INFO']
        callers = INFO.split(';')[1].replace('Callers=', '').split(',')
        for i in callers:
            sv.loc[index, 'num_callers'] = len(callers)
            sv.loc[index, i] = 1
    sv = sv[(sv['num_callers']>=2) & (sv['brass'] == 1)]

    cnv = pd.read_csv(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/uk_all_0115/{sample}/{sample}.subclone.final.cns",sep='\t')
    cnv['chromosome'] = cnv['chromosome'].astype(str)
    cnv = cnv[(cnv['cn1']!=1) | (cnv['cn2']!=1)]

    window = 1000

    for index, row in sv.iterrows():
        sv_chr = row['SV chrom']
        sv_start = row['SV start']
        sv_end = row['SV end']
        sv_length = row['SV length']
        for i, r in cnv.iterrows():
            cnv_chr = r['chromosome']
            cnv_start = r['start']
            cnv_end = r['end']
            if sv_chr == cnv_chr:
                if abs(sv_start - cnv_start)<window:
                    overlap = (min(cnv_end, sv_end) - max(cnv_start, sv_start))/sv_length
                    sv.loc[index, 'cnv'] = f"{cnv_chr}:{cnv_start}-{cnv_end} {overlap}"
    sv.to_csv(f"/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/integrate_cnv_in_sv/{sample}.tsv",sep='\t')

IID_H158671_T01_01_WG01
IID_H158670_T01_01_WG01
IID_H158669_T01_01_WG01
IID_H158668_T01_01_WG01


/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IID_H158667_T01_01_WG01
IID_H158666_T01_01_WG01


/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,38,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IID_H158665_T01_01_WG01
IID_H158664_T01_01_WG01
IID_H158663_T01_01_WG01
IID_H158662_T01_01_WG01


...

IID_H158629_T01_01_WG01
IID_H158628_T01_01_WG01
IID_H158627_T01_01_WG01
IID_H158626_T01_01_WG01
IID_H158625_T01_01_WG01
IID_H158624_T01_01_WG01
IID_H158623_T01_01_WG01
IID_H158622_T01_01_WG01
IID_H158621_T01_01_WG01


/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,38,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IID_H158620_T01_01_WG01
IID_H158619_T01_01_WG01
IID_H158618_T01_01_WG01
IID_H158617_T01_01_WG01
IID_H158616_T01_01_WG01
IID_H158615_T01_01_WG01
IID_H158614_T01_01_WG01
IID_H158613_T01_01_WG01
IID_H158612_T01_01_WG01


/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,38,61,63,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IID_H158611_T01_01_WG01
IID_H158610_T01_01_WG01
IID_H158609_T01_01_WG01
IID_H158608_T01_01_WG01
IID_H158607_T01_01_WG01
IID_H158606_T01_01_WG01
IID_H158605_T01_01_WG01
IID_H158604_T01_01_WG01


In [88]:
sv

,AnnotSV ID,SV chrom,SV start,SV end,SV length,SV type,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,I-H-108298-N1-1-D1-1,IID_H158606_T01_01_WG01,AnnotSV type,Gene name,NM,CDS length,tx length,location,intersectStart,intersectEnd,DGV_GAIN_IDs,DGV_GAIN_n_samples_with_SV,DGV_GAIN_n_samples_tested,DGV_GAIN_Frequency,DGV_LOSS_IDs,DGV_LOSS_n_samples_with_SV,DGV_LOSS_n_samples_tested,DGV_LOSS_Frequency,GD_ID,GD_AN,GD_N_HET,GD_N_HOMALT,GD_AF,GD_POPMAX_AF,GD_ID_others,DDD_SV,DDD_DUP_n_samples_with_SV,DDD_DUP_Frequency,DDD_DEL_n_samples_with_SV,DDD_DEL_Frequency,1000g_event,1000g_AF,1000g_max_AF,IMH_ID,IMH_AF,IMH_ID_others,promoters,dbVar_event,dbVar_variant,dbVar_status,TADcoordinates,ENCODEexperiments,GCcontent_left,GCcontent_right,Repeats_coord_left,Repeats_type_left,Repeats_coord_right,Repeats_type_right,ACMG,HI_CGscore,TriS_CGscore,DDD_status,DDD_mode,DDD_consequence,DDD_disease,DDD_pmids,HI_DDDpercent,synZ_ExAC,misZ_ExAC,pLI_ExAC,delZ_ExAC,dupZ_ExAC,cnvZ_ExAC,morbidGenes,morbidGenesCandidates,Mim Number,Phenotypes,Inheritance,AnnotSV ranking,num_callers,gridss,svaba,brass
15532,10_28821260_28848026_DEL,10,28821260,28848026,26766.0,DEL,.,N,N[10:28848026[,.,PASS,"END=28848026;Callers=brass,svaba,gridss;CHR2=1...",brass_IID_H158606_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::....:.:.:.:.:.:.:.:.:.:.:.:...,48:0:0.578947:0.571429:0.571429:51:-71.56:0:24...,full,WAC/WAC-AS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,-1.0,-1.0,NaN,-1.000000,10:8738346-108102195_INV;10:15431941-70315351_INV,WAC/WAC-AS1,NaN,NaN,NaN,NaN,NaN,0.650000,0.415000,NaN,NaN,10:28848065-28848094/10:28848118-28848410,(TG)n/L2a,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,4,3.0,1.0,1.0,1.0
25374,11_68782074_68787123_DEL,11,68782074,68787123,5049.0,DEL,.,N,N[11:68787123[,.,PASS,"END=68787123;Callers=brass,svaba,gridss;CHR2=1...",brass_IID_H158606_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::....:.:.:.:.:.:.:.:.:.:.:.:...,57:20:0.480519:0.5:0.45679:64:-89.38:0:31:88.4...,full,MRGPRF-AS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,-1.0,-1.0,NaN,-1.000000,11:1642919-71249653_INV;11:49233127-89368291_I...,NaN,NaN,NaN,NaN,NaN,NaN,0.515000,0.610000,11:68782152-68782459,MLT1A0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3.0,1.0,1.0,1.0
25495,11_77932829_77939621_DUP,11,77932829,77939621,6792.0,DUP,.,N,]11:77939621]N,.,PASS,"END=77939621;Callers=brass,svaba,gridss;CHR2=1...",brass_IID_H158606_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::....:.:.:.:.:.:.:.:.:.:.:.:...,21:7:1.19481:1.22667:1.17949:0:-73.68:21:10:72...,full,GAB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,-1.0,-1.0,11:77550802-78000858_DUP,0.000059,11:49233127-89368291_INV;11:49425031-89029881_...,NaN,NaN,NaN,NaN,NaN,NaN,0.575000,0.470000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.04,0.355310,-1.040969,0.997629,0.920190,0.780302,1.033942,NaN,NaN,NaN,NaN,NaN,2,3.0,1.0,1.0,1.0
35276,13_83435494_83439889_DEL,13,83435494,83439889,4395.0,DEL,.,N,N[13:83439889[,.,PASS,"END=83439889;Callers=brass,svaba,gridss;CHR2=1...",brass_IID_H158606_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::....:.:.:.:.:.:.:.:.:.:.:.:...,48:25:0.493506:0.487179:0.520548:66:-113.3:0:3...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,gnomAD_v2_DEL_13_161665;gnomAD_v2_DEL_13_161668,21476;18384,1;2,0;0,0.00010900000052060932,0.000720,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,-1.0,-1.0,13:83249177-83483973_DEL,0.000059,13:32957269-100763564_INV;13:33672321-10440911...,NaN,NaN,NaN,NaN,NaN,NaN,0.220000,0.280000,13:83435564-83435614,A-rich,13:83439503-83439802/13:83439899-83439941,AluSg/(TATG)n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3.0,

In [89]:
cnv

,chromosome,start,end,gene,log2,baf,cn,cn1,cn2,depth,probes,weight,aberrant_cell_frac
9,9,121152952,121294770,-,-0.772378,0.840909,1.0,1.0,0.0,45.1837,152,145.729000,1.0
24,X,1446744,2699498,-,-0.859047,0.540000,1.0,1.0,0.0,74.0274,1271,858.825000,1.0
26,8,143850648,143929135,-,0.583125,0.674376,3.0,2.0,1.0,121.6700,32,0.947261,NaN
27,9,121152837,121295287,-,-0.772284,0.842023,1.0,1.0,0.0,45.1754,23,0.958744,NaN
28,10,28821259,28848026,-,-0.787244,0.820922,1.0,1.0,0.0,44.3410,7,0.955477,NaN


In [85]:
window = 1000

for index, row in sv.iterrows():
    sv_chr = row['SV chrom']
    sv_start = row['SV start']
    sv_end = row['SV end']
    sv_length = row['SV length']
    for i, r in cnv.iterrows():
        cnv_chr = r['chromosome']
        cnv_start = r['start']
        cnv_end = r['end']
        if sv_chr == cnv_chr:
            if abs(sv_start - cnv_start)<window:
                overlap = (min(cnv_end, sv_end) - max(cnv_start, sv_start))/sv_length
                sv.loc[index, 'cnv'] = f"{cnv_chr}:{cnv_start}-{cnv_end} {overlap}"

In [86]:
sv[sv['cnv'].notnull()]

,AnnotSV ID,SV chrom,SV start,SV end,SV length,SV type,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,I-H-108298-N1-1-D1-1,IID_H158605_T01_01_WG01,AnnotSV type,Gene name,NM,CDS length,tx length,location,intersectStart,intersectEnd,DGV_GAIN_IDs,DGV_GAIN_n_samples_with_SV,DGV_GAIN_n_samples_tested,DGV_GAIN_Frequency,DGV_LOSS_IDs,DGV_LOSS_n_samples_with_SV,DGV_LOSS_n_samples_tested,DGV_LOSS_Frequency,GD_ID,GD_AN,GD_N_HET,GD_N_HOMALT,GD_AF,GD_POPMAX_AF,GD_ID_others,DDD_SV,DDD_DUP_n_samples_with_SV,DDD_DUP_Frequency,DDD_DEL_n_samples_with_SV,DDD_DEL_Frequency,1000g_event,1000g_AF,1000g_max_AF,IMH_ID,IMH_AF,IMH_ID_others,promoters,dbVar_event,dbVar_variant,dbVar_status,TADcoordinates,ENCODEexperiments,GCcontent_left,GCcontent_right,Repeats_coord_left,Repeats_type_left,Repeats_coord_right,Repeats_type_right,ACMG,HI_CGscore,TriS_CGscore,DDD_status,DDD_mode,DDD_consequence,DDD_disease,DDD_pmids,HI_DDDpercent,synZ_ExAC,misZ_ExAC,pLI_ExAC,delZ_ExAC,dupZ_ExAC,cnvZ_ExAC,morbidGenes,morbidGenesCandidates,Mim Number,Phenotypes,Inheritance,AnnotSV ranking,num_callers,gridss,svaba,brass,cnv
50275,5_60201995_60294535_DEL,5,60201995,60294535,92540.0,DEL,.,N,N[5:60294535[,.,PASS,"END=60294535;Callers=gridss,svaba,brass;CHR2=5...",brass_IID_H158605_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::...:.:.:.:.:.:.:.:.:.:.:.:....,47:22:0.493506:0.487179:0.475:53:-102.3:0:34:1...,full,ERCC8/NDUFAF2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.0,NaN,0,0,-1.0,gnomAD_v2_DEL_5_67953,21476,1,0,4.70000013592653e-05,0.000561,gnomAD_v2_DUP_5_15058,NaN,NaN,-1.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,-1.000000,5:60121012-60337042_DUP;5:24758284-101693420_I...,ERCC8/NDUFAF2,NaN,NaN,NaN,5:60200001-61080000;5:60200001-61120000;5:6020...,ENCFF066NYU:ENCFF075QYD:ENCFF136AUY:ENCFF248DG...,0.400000,0.345000,5:60201501-60201986/5:60202021-60202154,LTR8A/AluJo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.74,-0.648906,-0.460345,NaN,-1.661223,-0.234814,-0.860460,yes,NaN,NaN,NaN,NaN,4,3.0,1.0,1.0,1.0,5:60201994-60294535 1.0
56231,7_110382221_110493855_DEL,7,110382221,110493855,111634.0,DEL,.,N,N[7:110493855[,.,PASS,"END=110493855;Callers=gridss,svaba,brass;CHR2=...",brass_IID_H158605_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::...:.:.:.:.:.:.:.:.:.:.:.:....,33:27:0.506494:0.5:0.534247:38:-92.42:0:30:92....,full,IMMP2L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.0,NaN,0,0,-1.0,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.0,NaN,-1.0,NaN,-1.0,-1.0,7:110382216-110493864_DEL,0.000059,7:66631328-143298938_INV;7:79317852-147370690_...,NaN,NaN,NaN,NaN,7:110400001-111360000;7:110440001-111360000,ENCFF274VJU:ENCFF588KUZ:ENCFF701HCM:ENCFF996RJN,0.275000,0.280000,7:110382299-110382541,L1ME4a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.25,-0.543226,-0.023467,0.034132,-2.623544,0.194114,-2.471628,NaN,NaN,NaN,NaN,NaN,2,3.0,1.0,1.0,1.0,7:110382219-110493855 1.0
58404,9_15424528_22380856_DEL,9,15424528,22380856,6956328.0,DEL,.,N,N[9:22380856[,.,PASS,"END=22380856;Callers=gridss,svaba,brass;CHR2=9...",brass_IID_H158605_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::...:.:.:.:.:.:.:.:.:.:.:.:....,39:15:0.618421:0.618421:0.61039:50:-107.6:0:33...,full,ACER2/ADAMTSL1/BNC2/BNC2-AS1/C9orf92/CCDC171/C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.0,NaN,0,0,-1.0,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,-1.000000,NaN,ACER2/ADAMTSL1/BNC2/BNC2-AS1/C9orf92/CCDC171/C...,copy_number_gain;copy_number_loss,nssv13651853;nssv578326;nssv582133,Pathogenic,9:15680001-17560000;9:15720001-17000000;9:1572...,ENCFF032FMN:ENCFF066NYU:ENCFF075QYD:ENCFF136AU...,0.210000,0.400000,9:15424411-15424698,MER5C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,5,3.0,1.0,1.0,1.0,9:15424527-22380856 1.0
58477,9_21975806_22009649_DEL,9,21975806,22009649,33843.0,DEL,.,N,N[9:22009649[,.,PASS,"END=22009649;Callers=gridss,svaba,brass;CHR2=9...",brass_IID_H158605_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::...:.:.:.:.:.:.:.:.:.:.:.:....,37:15:0.210526:0.210526:0.347826:43:-79.22:0:2...,full,CDKN2A/

In [120]:
import pysam

def gridss_somatic(inp):
    outdir, name, path = inp
    """Filter gridss input to just somatic."""
    print(name)
     
    updated_inp = []
    new_path = os.path.join(outdir, name + "_gridss_somatic.vcf")
    with pysam.VariantFile(path, mode="r") as vcf_in:
        with pysam.VariantFile(
            new_path, mode="w", header=vcf_in.header
        ) as vcf_out:
            for rec in vcf_in:
                if not rec.info.get("SVTYPE"):
                    continue
                if rec.filter and not "PASS" in list(rec.filter):
                    continue
                if not ("[" in rec.alts[0] or "]" in rec.alts[0]):
                    continue
                if rec.samples[0]["QUAL"] > 0:
                    continue
                vcf_out.write(rec)
    updated_inp.append((name, new_path))
    return updated_inp


samples = [
     'IID_H158644_T01_01_WG01',
     'IID_H158643_T01_01_WG01',
     'IID_H158642_T01_01_WG01',
     'IID_H158641_T01_01_WG01',
     'IID_H158640_T01_01_WG01',
     'IID_H158639_T01_01_WG01',
     'IID_H158638_T01_01_WG01',
     'IID_H158637_T01_01_WG01',
     'IID_H158636_T01_01_WG01',
     'IID_H158635_T01_01_WG01',
     'IID_H158634_T01_01_WG01',
     'IID_H158633_T01_01_WG01',
     'IID_H158632_T01_01_WG01',
     'IID_H158631_T01_01_WG01',
     'IID_H158630_T01_01_WG01',
     'IID_H158629_T01_01_WG01',
     'IID_H158628_T01_01_WG01',
     'IID_H158627_T01_01_WG01',
     'IID_H158626_T01_01_WG01',
     'IID_H158625_T01_01_WG01',
     'IID_H158624_T01_01_WG01',
     'IID_H158623_T01_01_WG01',
     'IID_H158622_T01_01_WG01',
     'IID_H158621_T01_01_WG01',
     'IID_H158620_T01_01_WG01',
     'IID_H158619_T01_01_WG01',
     'IID_H158618_T01_01_WG01',
     'IID_H158617_T01_01_WG01',
     'IID_H158616_T01_01_WG01',
     'IID_H158615_T01_01_WG01',
     'IID_H158614_T01_01_WG01',
     'IID_H158613_T01_01_WG01',
     'IID_H158612_T01_01_WG01',
     'IID_H158611_T01_01_WG01',
     'IID_H158610_T01_01_WG01',
     'IID_H158609_T01_01_WG01',
     'IID_H158608_T01_01_WG01',
     'IID_H158607_T01_01_WG01',
     'IID_H158606_T01_01_WG01',
     'IID_H158605_T01_01_WG01',
     'IID_H158604_T01_01_WG01'
]
outdir = "/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/gridss_somatic"

import multiprocessing
from functools import partial


inp = []
pool = multiprocessing.Pool(4)
pool.map(gridss_somatic, [[outdir, sample, f"/work/isabl/home/gutierj2/uk_all/GRIDDS_unmatched/universal_normal/joe/GRIDSS/{sample}_unmatched.sv.vcf"] for sample in samples])



IID_H158638_T01_01_WG01
IID_H158635_T01_01_WG01
IID_H158644_T01_01_WG01
IID_H158641_T01_01_WG01
IID_H158640_T01_01_WG01
IID_H158637_T01_01_WG01
IID_H158634_T01_01_WG01
IID_H158643_T01_01_WG01
IID_H158639_T01_01_WG01
IID_H158636_T01_01_WG01
IID_H158633_T01_01_WG01
IID_H158642_T01_01_WG01
IID_H158632_T01_01_WG01
IID_H158629_T01_01_WG01
IID_H158626_T01_01_WG01
IID_H158623_T01_01_WG01
IID_H158631_T01_01_WG01
IID_H158628_T01_01_WG01
IID_H158625_T01_01_WG01
IID_H158622_T01_01_WG01
IID_H158630_T01_01_WG01
IID_H158627_T01_01_WG01
IID_H158621_T01_01_WG01
IID_H158624_T01_01_WG01
IID_H158620_T01_01_WG01
IID_H158617_T01_01_WG01
IID_H158614_T01_01_WG01
IID_H158611_T01_01_WG01
IID_H158616_T01_01_WG01
IID_H158619_T01_01_WG01
IID_H158613_T01_01_WG01
IID_H158610_T01_01_WG01
IID_H158615_T01_01_WG01
IID_H158612_T01_01_WG01
IID_H158618_T01_01_WG01
IID_H158609_T01_01_WG01
IID_H158608_T01_01_WG01
IID_H158605_T01_01_WG01
IID_H158604_T01_01_WG01
IID_H158607_T01_01_WG01
IID_H158606_T01_01_WG01


[[('IID_H158644_T01_01_WG01',
   '/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/gridss_somatic/IID_H158644_T01_01_WG01_gridss_somatic.vcf')],
 [('IID_H158643_T01_01_WG01',
   '/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/gridss_somatic/IID_H158643_T01_01_WG01_gridss_somatic.vcf')],
 [('IID_H158642_T01_01_WG01',
   '/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/gridss_somatic/IID_H158642_T01_01_WG01_gridss_somatic.vcf')],
 [('IID_H158641_T01_01_WG01',
   '/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/gridss_somatic/IID_H158641_T01_01_WG01_gridss_somatic.vcf')],
 [('IID_H158640_T01_01_WG01',
   '/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/gridss_somatic/IID_H158640_T01_01_WG01_gridss_somatic.vcf')],
 [('IID_H158639_T01_01_WG01',
   '/work/isabl/home/zhouy1/un

Process ForkPoolWorker-51:
Process ForkPoolWorker-52:
Process ForkPoolWorker-50:
Process ForkPoolWorker-49:
Traceback (most recent call last):
  File "/work/isabl/opt/python/v3.6.5/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/work/isabl/opt/python/v3.6.5/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/work/isabl/opt/python/v3.6.5/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/work/isabl/opt/python/v3.6.5/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/work/isabl/opt/python/v3.6.5/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/work/isabl/opt/python/v3.6.5/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/work/

In [126]:
sv = pd.read_csv("/work/isabl/home/zhouy1/unmatechedSVpipeline/test/unmatched_benchmark/sv_triplecaller_merge/annotated/no_normal/IID_H158671_T01_01_WG01/IID_H158671_T01_01_WG01.merged.annotated.tsv",sep='\t')

sv = sv[sv['AnnotSV type']=='full']
sv['SV chrom'] = sv['SV chrom'].astype(str)
sv = sv[(sv['SV type']=='DEL') | (sv['SV type']=='DUP')]
for index, row in sv.iterrows():
    INFO = row['INFO']
    callers = INFO.split(';')[1].replace('Callers=', '').split(',')
    for i in callers:
        sv.loc[index, 'num_callers'] = len(callers)
        sv.loc[index, i] = 1
sv = sv[(sv['num_callers']>=2) & (sv['brass'] == 1)]
sv

/juno/work/isabl/opt/python/.virtualenvs/users/zhouy1/jupyter-python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,AnnotSV ID,SV chrom,SV start,SV end,SV length,SV type,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,I-H-108298-N1-1-D1-1,IID_H158671_T01_01_WG01,AnnotSV type,Gene name,NM,CDS length,tx length,location,intersectStart,intersectEnd,DGV_GAIN_IDs,DGV_GAIN_n_samples_with_SV,DGV_GAIN_n_samples_tested,DGV_GAIN_Frequency,DGV_LOSS_IDs,DGV_LOSS_n_samples_with_SV,DGV_LOSS_n_samples_tested,DGV_LOSS_Frequency,GD_ID,GD_AN,GD_N_HET,GD_N_HOMALT,GD_AF,GD_POPMAX_AF,GD_ID_others,DDD_SV,DDD_DUP_n_samples_with_SV,DDD_DUP_Frequency,DDD_DEL_n_samples_with_SV,DDD_DEL_Frequency,1000g_event,1000g_AF,1000g_max_AF,IMH_ID,IMH_AF,IMH_ID_others,promoters,dbVar_event,dbVar_variant,dbVar_status,TADcoordinates,ENCODEexperiments,GCcontent_left,GCcontent_right,Repeats_coord_left,Repeats_type_left,Repeats_coord_right,Repeats_type_right,ACMG,HI_CGscore,TriS_CGscore,DDD_status,DDD_mode,DDD_consequence,DDD_disease,DDD_pmids,HI_DDDpercent,synZ_ExAC,misZ_ExAC,pLI_ExAC,delZ_ExAC,dupZ_ExAC,cnvZ_ExAC,morbidGenes,morbidGenesCandidates,Mim Number,Phenotypes,Inheritance,AnnotSV ranking,num_callers,svaba,gridss,brass
16930,10_111768609_111825249_DEL,10,111768609,111825249,56640.0,DEL,.,N,N[10:111825249[,.,PASS,"END=111825249;Callers=brass,svaba,gridss;CHR2=...",brass_IID_H158671_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::...:.:.:.:.:.:.:.:.:.:.:.:....,27:0:0.71875:0.71875:0.71875:33:-80.81:26:0:77...,full,ADD3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.000000,NaN,-1.00000,NaN,-1.0,-1.0,NaN,-1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.435000,0.400000,NaN,NaN,10:111825232-111825294/10:111825297-111825597,CT-rich/AluSz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.15,-0.277842,-0.395241,0.976285,1.051922,1.237284,1.477487,yes,NaN,NaN,NaN,NaN,4,3.0,1.0,1.0,1.0
29180,12_69547381_118732981_DUP,12,69547381,118732981,49185600.0,DUP,.,N,]12:118732981]N,.,PASS,"END=118732981;Callers=brass,svaba,gridss;CHR2=...",brass_IID_H158671_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::....:.:.:.:.:.:.:.:.:.:.:.:...,"17:0:1:1:1.01515:0:-204.7:52:27:204.6,18.6,0:1...",full,ACACB/ACAD10/ACSS3/ACTR6/ADAM1A/ALDH1L2/ALDH2/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,gnomAD_v2_INV_12_581,NaN,NaN,-1.000000,NaN,-1.00000,NaN,-1.0,-1.0,NaN,-1.00000,12:40933233-113477160_INV;12:70180800-11032003...,ACACB/ACAD10/ACSS3/ACTR6/ADAM1A/ALDH1L2/ALDH2/...,copy_number_gain;copy_number_loss;deletion,nssv13638540;nssv13639325;nssv13640048;nssv136...,Likely pathogenic;Pathogenic;Uncertain signifi...,12:100120001-101160000;12:100120001-101200000;...,ENCFF032FMN:ENCFF066NYU:ENCFF075QYD:ENCFF136AU...,0.480000,0.540000,12:69547372-69547558,AluSp,12:118732861-118733143,AluSc,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,5,3.0,1.0,1.0,1.0
32022,13_41550346_41592097_DEL,13,41550346,41592097,41751.0,DEL,.,N,N[13:41592097[,.,PASS,"END=41592097;Callers=brass,svaba,gridss;CHR2=1...",brass_IID_H158671_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::...:.:.:.:.:.:.:.:.:.:.:.:....,45:14:0.552239:0.544118:0.513889:48:-89.12:0:2...,full,ELF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,13:41550804-41593694,2.0,0.002367,0.0,0.00000,NaN,-1.0,-1.0,NaN,-1.00000,13:28522573-70820752_INV;13:32436828-48860329_...,ELF1,NaN,NaN,NaN,NaN,NaN,0.355000,0.370000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.11,0.021558,-0.094159,0.395702,-1.516257,1.081920,-0.124889,NaN,NaN,NaN,NaN,NaN,2,3.0,1.0,1.0,1.0
33751,14_65542483_65567802_DEL,14,65542483,65567802,25319.0,DEL,.,N,N[14:65567802[,.,PASS,"END=65567802;Callers=brass,svaba,gridss;CHR2=1...",brass_IID_H158671_T01_01_WG01_PS:brass_IID_H15...,0:0:.:.:.:.:.:.:.::....:.:.:.:.:.:.:.:.:.:.:.:...,35:11:0.567164:0.575758:0.527778:38:-158.4:34:...,full,LOC100506321/MAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1.000000,NaN,0,0,-1.000000,NaN,NaN,NaN,NaN,-1,-1.000000,NaN,NaN,NaN,-1.000000,NaN,-1.00000,NaN,-1.0,-1.0,NaN,-1.00000,14:21128